In [1]:
#%load_ext watermark
%load_ext autoreload 
%autoreload 2

In [2]:
import os
import sys
from copy import deepcopy
import itertools
import numpy as np
import pandas as pd
from tqdm import tqdm
from subprocess import call
# import pybpr
import matplotlib.pyplot as plt
from functools import partial
from sklearn.feature_extraction import DictVectorizer
from scipy.special import expit

In [4]:
from scipy.sparse import csr_matrix, dok_matrix, spmatrix

In [3]:
data_dir = '/home/rsandhu/zazzle/rsandhu/pybpr/examples/data/netflix'
df_items = pd.read_csv(os.path.join(data_dir, 'netflix_titles.csv'))

In [16]:
from pybpr import *

ModuleNotFoundError: No module named 'pybpr'

In [4]:
df_items.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8807 entries, 0 to 8806
Data columns (total 12 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   show_id       8807 non-null   object
 1   type          8807 non-null   object
 2   title         8807 non-null   object
 3   director      6173 non-null   object
 4   cast          7982 non-null   object
 5   country       7976 non-null   object
 6   date_added    8797 non-null   object
 7   release_year  8807 non-null   int64 
 8   rating        8803 non-null   object
 9   duration      8804 non-null   object
 10  listed_in     8807 non-null   object
 11  description   8807 non-null   object
dtypes: int64(1), object(11)
memory usage: 825.8+ KB


In [5]:
attributes = df_items.listed_in.unique().tolist()
attributes = [ix.split(', ') for ix in attributes]
attributes = list(set([iy for ix in attributes for iy in ix]))
#print(len(attributes) == len(set(attributes)))
print(len(attributes))
for ix in attributes:
    print(ix)

42
International Movies
LGBTQ Movies
Docuseries
International TV Shows
TV Sci-Fi & Fantasy
Spanish-Language TV Shows
Action & Adventure
Horror Movies
Children & Family Movies
Crime TV Shows
Classic Movies
Anime Series
TV Thrillers
Faith & Spirituality
TV Horror
British TV Shows
Dramas
Comedies
Anime Features
TV Dramas
Science & Nature TV
TV Comedies
Korean TV Shows
Music & Musicals
TV Shows
Stand-Up Comedy & Talk Shows
Romantic Movies
Thrillers
TV Action & Adventure
Teen TV Shows
Sports Movies
Cult Movies
Classic & Cult TV
Documentaries
Kids' TV
Romantic TV Shows
Sci-Fi & Fantasy
Stand-Up Comedy
Movies
Reality TV
TV Mysteries
Independent Movies


In [6]:
ilist = df_items.listed_in.tolist()
ilist= [{ix:1. for ix in iy.split(', ')} for iy in ilist]
ilist[:5]

[{'Documentaries': 1.0},
 {'International TV Shows': 1.0, 'TV Dramas': 1.0, 'TV Mysteries': 1.0},
 {'Crime TV Shows': 1.0,
  'International TV Shows': 1.0,
  'TV Action & Adventure': 1.0},
 {'Docuseries': 1.0, 'Reality TV': 1.0},
 {'International TV Shows': 1.0, 'Romantic TV Shows': 1.0, 'TV Comedies': 1.0}]

In [7]:
v = DictVectorizer(sparse=False)
Xmat = v.fit_transform(ilist)
Xmat.shape

(8807, 42)

In [76]:
Cmat = Xmat.T @ Xmat 
Cmat = np.abs(np.round(np.corrcoef(Cmat),2))
print(Cmat[13,:])
np.argwhere(Cmat == np.min(Cmat))

[0.19 0.05 0.09 0.12 0.16 0.13 0.22 0.26 0.12 0.05 0.16 0.1  0.54 1.
 0.   0.33 0.41 0.13 0.08 0.11 0.22 0.05 0.2  0.09 0.3  0.11 0.07 0.1
 0.11 0.22 0.05 0.09 0.13 0.11 0.11 0.13 0.12 0.12 0.05 0.12 0.12 0.21]


array([[ 4, 10],
       [10,  4],
       [11, 40],
       [13, 14],
       [14, 13],
       [18, 25],
       [18, 34],
       [25, 18],
       [31, 39],
       [34, 18],
       [39, 31],
       [40, 11]])

In [77]:
Cmat = Xmat.T @ Xmat 
iz = 13
print(v.feature_names_[iz])
for ix, iy in zip(Cmat[iz,:],v.feature_names_):
    print(int(ix), ' : ', iy)

Faith & Spirituality
3  :  Action & Adventure
0  :  Anime Features
0  :  Anime Series
0  :  British TV Shows
9  :  Children & Family Movies
0  :  Classic & Cult TV
0  :  Classic Movies
8  :  Comedies
0  :  Crime TV Shows
0  :  Cult Movies
12  :  Documentaries
0  :  Docuseries
44  :  Dramas
65  :  Faith & Spirituality
0  :  Horror Movies
1  :  Independent Movies
18  :  International Movies
0  :  International TV Shows
0  :  Kids' TV
0  :  Korean TV Shows
0  :  LGBTQ Movies
0  :  Movies
2  :  Music & Musicals
0  :  Reality TV
4  :  Romantic Movies
0  :  Romantic TV Shows
1  :  Sci-Fi & Fantasy
0  :  Science & Nature TV
0  :  Spanish-Language TV Shows
1  :  Sports Movies
0  :  Stand-Up Comedy
0  :  Stand-Up Comedy & Talk Shows
0  :  TV Action & Adventure
0  :  TV Comedies
0  :  TV Dramas
0  :  TV Horror
0  :  TV Mysteries
0  :  TV Sci-Fi & Fantasy
0  :  TV Shows
0  :  TV Thrillers
0  :  Teen TV Shows
0  :  Thrillers


In [79]:
np.where(Xmat[:,13] == 1)

(array([ 171,  576,  774,  834, 1107, 1156, 1351, 1366, 1474, 2092, 2119,
        2380, 2622, 2887, 2951, 3164, 3413, 3556, 4160, 4161, 4765, 4928,
        4937, 5058, 5132, 5953, 6010, 6157, 6312, 6438, 6476, 6539, 6637,
        6710, 6775, 6802, 6858, 6861, 6862, 6986, 6987, 7049, 7127, 7281,
        7304, 7412, 7533, 7578, 7652, 7754, 7928, 7940, 8059, 8239, 8247,
        8326, 8327, 8328, 8329, 8339, 8346, 8381, 8400, 8562, 8639]),)

In [80]:
np.where(Xmat[:,14] == 1)

(array([  28,   42,   43,   44,  200,  210,  211,  229,  236,  237,  238,
         239,  299,  307,  332,  353,  396,  398,  441,  463,  479,  550,
         556,  557,  586,  607,  610,  611,  612,  643,  728,  740,  796,
         804,  822,  836,  853,  887,  929,  962,  964,  996, 1027, 1041,
        1087, 1100, 1118, 1119, 1139, 1177, 1222, 1283, 1284, 1297, 1367,
        1398, 1447, 1481, 1536, 1629, 1637, 1648, 1652, 1653, 1714, 1775,
        1777, 1783, 1784, 1788, 1806, 1816, 1843, 1867, 1879, 1898, 1900,
        1970, 2029, 2036, 2091, 2104, 2149, 2342, 2382, 2443, 2463, 2464,
        2477, 2589, 2705, 2729, 2749, 2889, 2908, 2933, 2968, 3014, 3060,
        3077, 3097, 3102, 3162, 3262, 3379, 3384, 3398, 3421, 3434, 3450,
        3568, 3569, 3570, 3638, 3639, 3698, 3729, 3757, 3768, 3800, 3873,
        3919, 3922, 3923, 3930, 3948, 3959, 3972, 4059, 4133, 4147, 4165,
        4191, 4194, 4195, 4211, 4241, 4277, 4374, 4375, 4382, 4392, 4395,
        4510, 4512, 4515, 4524, 4528, 

In [37]:
#initiate
n_features = len(attributes)
n_items = df_items.shape[0]
n_latent = 20
item_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(n_items+n_features,n_latent))
print(item_latent_mat.shape)
#np.round(item_latent_mat,2)

(8849, 20)


In [38]:
from sklearn.decomposition import PCA
pca = PCA(n_components=n_latent).fit(Xmat)
#pca_vecs = np.round(pca.components_,3)
pca.components_.T.shape

(42, 20)

In [39]:
item_latent_mat[n_items:,:] = pca.components_.T
np.round(np.linalg.norm(item_latent_mat, axis=1),2)

array([0.39, 0.48, 0.3 , ..., 0.05, 0.05, 0.99])

In [41]:
for ix in range(n_items):
    item_latent_mat[ix,:] /= 10*np.linalg.norm(item_latent_mat[ix,:])
for ix in range(n_features):
    item_latent_mat[n_items+ix,:] /= np.linalg.norm(item_latent_mat[n_items+ix,:])
np.round(np.linalg.norm(item_latent_mat, axis=1),2)

array([0.1, 0.1, 0.1, ..., 1. , 1. , 1. ])

In [116]:
n_users = 1000
user_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(n_users,n_latent))
for ix in range(user_latent_mat.shape[0]):
    user_latent_mat[ix,:] /= 10*np.linalg.norm(user_latent_mat[ix,:])
print(user_latent_mat.shape)

(1000, 20)


In [117]:
# for user_u in range(n_users):
#     print('Recs for user: ', user_u)
#     rval = []
#     rec = user_latent_mat[user_u].dot(item_latent_mat.T)
#     for i in range(n_items):
#         inds = list(np.nonzero(Xmat[i,:])[0])
#         rval.append(np.sum(rec[inds])) 
#     item_inds = np.argsort(rval)[::-1][:5]
#     print(df_items.title.iloc[item_inds].values.tolist())

In [148]:
# interaction
int_users = np.repeat(list(range(n_users)),2)
int_items = np.tile([28,171], n_users)
# for ix, iy in zip(int_users, int_items):
#     print(f'User-{ix}', df_items[['title','listed_in']].iloc[iy].values)

In [149]:
#gradients
for ix in tqdm(range(100000)): 
    # pos-neg pairs
    user_u = np.random.choice(int_users)
    item_pos = int_items[user_u]
    item_neg = deepcopy(item_pos)
    while item_neg == item_pos:
        item_neg = np.random.choice(n_items)
    #print(f'::::::::Iter{ix}')
    # slice user-item latent mat
    user_latent = user_latent_mat[user_u]
    #print(np.round(user_latent,2))
    pos_inds = list(np.nonzero(Xmat[item_pos,:])[0])#item_df.iloc[item_pos].values.nonzero()[0]
    item_latent_pos = item_latent_mat[pos_inds,:]
    #print(np.round(item_latent_pos,2))
    item_latent_pos = np.sum(item_latent_pos,axis=0)
    #print(np.round(item_latent_pos,2))
    neg_inds = list(np.nonzero(Xmat[item_neg,:])[0]) #item_df.iloc[item_neg].values.nonzero()[0]
    item_latent_neg = item_latent_mat[neg_inds,:]
    #print(np.round(item_latent_neg,2))
    item_latent_neg = np.sum(item_latent_neg, axis=0)
    #print(np.round(item_latent_neg,2))

    r_uij = np.dot(user_latent, (item_latent_pos - item_latent_neg))
    sigmoid = expit(-r_uij)
    sigmoid_tiled = np.tile(sigmoid, (n_latent,))
    grad_u = np.multiply(sigmoid_tiled, (item_latent_neg - item_latent_pos))
    grad_pos = np.multiply(sigmoid_tiled, -user_latent)
    grad_neg = np.multiply(sigmoid_tiled, user_latent)

    # bpr update
    learning_rate=0.05
    user_latent_mat[user_u] -= learning_rate * grad_u
    item_latent_mat[item_pos] -= learning_rate * grad_pos
    item_latent_mat[item_neg] -= learning_rate * grad_neg
    #print('User:\n', np.round(user_latent_mat,3))
    #print('Item:\n', np.round(item_latent_mat,3))

100%|██████████| 100000/100000 [00:03<00:00, 25446.31it/s]


In [150]:
user_u = 0
rval = []
rec = user_latent_mat[user_u].dot(item_latent_mat.T)
for i in range(n_items):
    inds = list(np.nonzero(Xmat[i,:])[0])
    rval.append(np.sum(rec[inds])) 
item_inds = np.argsort(rval)[::-1][:20]
recs = df_items[['title','listed_in']].iloc[item_inds].values
for i in recs:
    print(i[0], ': ', i[1])

El Chema :  Crime TV Shows, Spanish-Language TV Shows, TV Action & Adventure
El Chapo :  Crime TV Shows, Spanish-Language TV Shows, TV Action & Adventure
Camelia la Texana :  Crime TV Shows, Romantic TV Shows, Spanish-Language TV Shows
Creators :  Kids' TV, Spanish-Language TV Shows
Heidi, bienvenida a casa :  Kids' TV, Spanish-Language TV Shows
Pocoyo :  Kids' TV, Spanish-Language TV Shows
Legend Quest: Masters of Myth :  Kids' TV, Spanish-Language TV Shows
Silvana Sin Lana :  Romantic TV Shows, Spanish-Language TV Shows, TV Comedies
Enemigo íntimo :  Crime TV Shows, Spanish-Language TV Shows, TV Dramas
Falsa identidad :  Crime TV Shows, Spanish-Language TV Shows, TV Dramas
Ultimate Beastmaster México :  Reality TV, Spanish-Language TV Shows
Go! Live Your Way :  Kids' TV, Spanish-Language TV Shows, Teen TV Shows
BREAK IT ALL: The History of Rock in Latin America :  Docuseries, Spanish-Language TV Shows
Maradona in Mexico :  Docuseries, Spanish-Language TV Shows
Street Food: Latin Amer

In [ ]:
item_df = pd.DataFrame.from_dict(dict(
    item1=[1,0,0,1],
    item2=[1,0,0,0],
    item3=[0,1,1,0],
    item4=[0,0,1,0],
    item5=[0,1,0,0]
), orient='index', columns=[f'feature{i+1}' for i in range(4)])
n_items = item_df.shape[0]
n_features = item_df.shape[1]
print(n_items, n_features)
item_df

In [ ]:
item_df_identity = pd.DataFrame(np.eye(item_df.shape[0]), dtype=int, index=item_df.index, columns=item_df.index)
item_df = pd.concat([item_df_identity, item_df], axis=1)
item_df

In [ ]:
user_df = pd.DataFrame(np.eye(3), dtype=int, index=[f'user{i+1}' for i in range(3)])
user_df.columns = list(user_df.index.values)
n_users = user_df.shape[0]
user_df

In [ ]:
# int_df = pd.DataFrame.from_dict(dict(
#     user1=[1,0,0,0,0],
#     user2=[0,0,1,0,0],
#     #user3=[1,0,1,1,0],
# ), orient='index', columns=[f'item{i+1}' for i in range(5)])
# int_df

In [ ]:
#initiate
nlatent = 2
user_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(user_df.shape[1],nlatent))
item_latent_mat = np.random.normal(loc=0.,scale=0.1,size=(item_df.shape[1],nlatent))
print(item_latent_mat.shape, user_latent_mat.shape)
np.round(item_latent_mat,2)

In [ ]:
item_df.values[:, item_df.shape[0]:]

In [ ]:
from sklearn.decomposition import PCA
pca = PCA(n_components=nlatent).fit(item_df.values[:, item_df.shape[0]:])
pca_vecs = np.round(pca.components_,3)
pca_vecs.T

In [ ]:
np.round(np.dot(pca_vecs[0,:], pca_vecs[1,:]),2)

In [ ]:
item_latent_mat[item_df.shape[0]:,:] = pca.components_.T
np.round(item_latent_mat,2)

In [ ]:
np.round(np.linalg.norm(item_latent_mat, axis=1),2)

In [ ]:
for ix in range(n_items):
    item_latent_mat[ix,:] /= 100*np.linalg.norm(item_latent_mat[ix,:])
for ix in range(n_features):
    item_latent_mat[n_items+ix,:] /= np.linalg.norm(item_latent_mat[n_items+ix,:])
for ix in range(user_latent_mat.shape[0]):
    user_latent_mat[ix,:] /= 10*np.linalg.norm(user_latent_mat[ix,:])
np.round(np.linalg.norm(item_latent_mat, axis=1),2)
#np.round(np.linalg.norm(user_latent_mat, axis=1),2)

In [ ]:
np.round(item_latent_mat,2)

In [ ]:
np.round(user_latent_mat,2)

In [ ]:
for user_u in range(n_users):
    print('Recs for user: ', user_u)
    rec = user_latent_mat[user_u].dot(item_latent_mat.T)
    for i in range(n_items):
        inds = item_df.iloc[i].values.nonzero()[0]
        print(i, ':', np.round(np.sum(rec[inds]),3))

In [ ]:
# interaction
user_u = 0
item_pos = 0
item_neg = 1

In [ ]:
#gradients
for ix in range(10): 
    print(f'::::::::Iter{ix}')
    # slice user-item latent mat
    user_latent = user_latent_mat[user_u]
    #print(np.round(user_latent,2))
    pos_inds = item_df.iloc[item_pos].values.nonzero()[0]
    item_latent_pos = item_latent_mat[pos_inds,:]
    #print(np.round(item_latent_pos,2))
    item_latent_pos = np.sum(item_latent_pos,axis=0)
    #print(np.round(item_latent_pos,2))
    neg_inds = item_df.iloc[item_neg].values.nonzero()[0]
    item_latent_neg = item_latent_mat[neg_inds,:]
    #print(np.round(item_latent_neg,2))
    item_latent_neg = np.sum(item_latent_neg, axis=0)
    #print(np.round(item_latent_neg,2))

    r_uij = np.dot(user_latent, (item_latent_pos - item_latent_neg))
    sigmoid = expit(-r_uij)
    sigmoid_tiled = np.tile(sigmoid, (nlatent,))
    grad_u = np.multiply(sigmoid_tiled, (item_latent_neg - item_latent_pos))
    grad_pos = np.multiply(sigmoid_tiled, -user_latent)
    grad_neg = np.multiply(sigmoid_tiled, user_latent)

    # bpr update
    learning_rate=0.1
    user_latent_mat[user_u] -= learning_rate * grad_u
    item_latent_mat[item_pos] -= learning_rate * grad_pos
    item_latent_mat[item_neg] -= learning_rate * grad_neg
    print('User:\n', np.round(user_latent_mat,3))
    print('Item:\n', np.round(item_latent_mat,3))
    rec = user_latent_mat[user_u].dot(item_latent_mat.T)
    for i in range(n_items):
        inds = item_df.iloc[i].values.nonzero()[0]
        print(i, ':', np.round(np.sum(rec[inds]),3))
    #print('Recs:', np.round(rec,3))